In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import MySQLdb
import time

from sympy import *
from sympy.solvers import solve
from sympy.parsing.sympy_parser import parse_expr


In [5]:
j = 0
d = 31

time_a = pd.date_range('2016-10-01 00:00:00', periods=96*d, freq='15T')
time_b = pd.date_range('2016-10-01 00:14:59', periods=96*d, freq='15T')

conn = MySQLdb.connect("localhost","conn","hems")
c = conn.cursor()

query_empty = """truncate table thesis.pv1200batt3""" 
c.execute(query_empty)
print "thesis.pv1200batt3 are Empty!!! \n"

query1 = "SELECT Timestamp,total_watt  FROM thesis.load WHERE  Timestamp >= '%s'   AND Timestamp <= '%s'; " % (time_a[0],time_b[(96*d)-1])
df1 = pd.read_sql(query1, conn)

query = "SELECT Timestamp,PV FROM thesis.PV WHERE Timestamp>='%s' AND Timestamp<='%s'; "%('2016-10-01 00:00:00','2016-10-31 23:59:59')
df = pd.read_sql(query, conn)
df['PV'] = ((df.PV/10)*4)

batt = float(2.9*1000)
minbatt = batt * 0.2
maxbatt = batt
levelbatt = []
levelbatt.append(float(batt))
pv_watt = []
batt_watt = []
grid_watt = []
effi = float(0.95)
pv_consumption = []
batt_consumption = []
grid_consumption = []

overpv = float(0)
sum_overpv = float(0)
charging_watt = float(0)

while ( j <= (96*d) -1):
    print j,"at time",time_b[j]
    if (df.PV[j] != 0):
        if (levelbatt[j] == maxbatt):
            batt_t = levelbatt[j] - minbatt
            watt = df1.total_watt[j] 
            if (df.PV[j] > watt):
                pv_consumption.append(float(watt))
                batt_consumption.append(float(0))
                grid_consumption.append(float(0))
                levelbatt.append(float(levelbatt[j]))
                overpv = df.PV[j] - watt
                sum_overpv += overpv
            else:
                lefted_pv = watt-df.PV[j]
                pv_consumption.append(float(df.PV[j]))
                if (batt_t > lefted_pv):
                    batt_consumption.append(float(lefted_pv))
                    grid_consumption.append(float(0))
                    levelbatt.append(float(levelbatt[j]-lefted_pv))
                else:
                    lefted_batt = lefted_pv - batt_t
                    batt_consumption.append(float(batt_t))
                    grid_consumption.append(float(lefted_batt))
                    levelbatt.append(float(levelbatt[j]-batt_t))                                          
        else:
            batt_t = levelbatt[j] - minbatt
            watt = df1.total_watt[j] 
            if (df.PV[j] > watt):
                pv_consumption.append(float(watt))
                batt_consumption.append(float(0))
                grid_consumption.append(float(0))
                charging_watt = df.PV[j] - watt
                missing_watt = maxbatt - levelbatt[j]
                if (charging_watt > missing_watt):
                    charging_watt = missing_watt *effi
                else:
                    charging_watt = charging_watt * effi 
                levelbatt.append(float(levelbatt[j]+(charging_watt)))
                overpv = df.PV[j] - watt - charging_watt
                sum_overpv += overpv   
            else:
                lefted_pv = watt-df.PV[j]
                pv_consumption.append(float(df.PV[j]))
                if (batt_t > lefted_pv):
                    batt_consumption.append(float(lefted_pv))
                    grid_consumption.append(float(0))
                    levelbatt.append(float(levelbatt[j]-lefted_pv))
                else:
                    lefted_batt = lefted_pv - batt_t
                    batt_consumption.append(float(batt_t))
                    grid_consumption.append(float(lefted_batt))
                    levelbatt.append(float(levelbatt[j]-batt_t))                        
    else :
        if (levelbatt[j] > minbatt):
            batt_t = levelbatt[j] - minbatt
            watt = df1.total_watt[j] 
            if (batt_t > watt):
                pv_consumption.append(float(0))
                batt_consumption.append(float(watt))
                grid_consumption.append(float(0))
                levelbatt.append(float(levelbatt[j]-watt))
            else:
                lefted_batt = watt - batt_t
                pv_consumption.append(float(0))
                batt_consumption.append(float(batt_t))
                grid_consumption.append(float(lefted_batt))
                levelbatt.append(float(levelbatt[j]-batt_t))          
        else:
            watt = df1.total_watt[j] 
            pv_consumption.append(float(0))
            batt_consumption.append(float(0))
            grid_consumption.append(float(watt))
            levelbatt.append(float(levelbatt[j]))
       
    c.execute("INSERT INTO  thesis.pv1200batt3 (Timestamp,total_watt ,PV_genarator , levelbatt ,charging_watt, pv_consumption , batt_consumption,grid_consumption) VALUES ( %s,  %s,%s, %s, %s, %s, %s, %s)",
              (time_b[j],df1.total_watt[j],df.PV[j],levelbatt[j],charging_watt,pv_consumption[j],batt_consumption[j],grid_consumption[j]))
    conn.commit()  
    charging_watt = float(0) 
    j +=1 
print "-------------YOUR PROGRAM IS DONE-----------------"

thesis.pv1200batt3 are Empty!!! 

0 at time 2016-10-01 00:14:59
1 at time 2016-10-01 00:29:59
2 at time 2016-10-01 00:44:59
3 at time 2016-10-01 00:59:59
4 at time 2016-10-01 01:14:59
5 at time 2016-10-01 01:29:59
6 at time 2016-10-01 01:44:59
7 at time 2016-10-01 01:59:59
8 at time 2016-10-01 02:14:59
9 at time 2016-10-01 02:29:59
10 at time 2016-10-01 02:44:59
11 at time 2016-10-01 02:59:59
12 at time 2016-10-01 03:14:59
13 at time 2016-10-01 03:29:59
14 at time 2016-10-01 03:44:59
15 at time 2016-10-01 03:59:59
16 at time 2016-10-01 04:14:59
17 at time 2016-10-01 04:29:59
18 at time 2016-10-01 04:44:59
19 at time 2016-10-01 04:59:59
20 at time 2016-10-01 05:14:59
21 at time 2016-10-01 05:29:59
22 at time 2016-10-01 05:44:59
23 at time 2016-10-01 05:59:59
24 at time 2016-10-01 06:14:59
25 at time 2016-10-01 06:29:59
26 at time 2016-10-01 06:44:59
27 at time 2016-10-01 06:59:59
28 at time 2016-10-01 07:14:59
29 at time 2016-10-01 07:29:59
30 at time 2016-10-01 07:44:59
31 at time 2016

In [6]:
print sum_overpv /1000

17.3769864794
